In [1]:
using Plots;plotly()
using Optim
using Zygote
using Meteor.Utility
using Meteor.ExactDiag
using DataFrames;using CSV
using Pkg

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/nudt/.julia/packages/Plots/kyYZF/src/backends.jl:372


In [2]:
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)

In [3]:
using Base.Threads;nthreads()

1

In [4]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end


function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν=1
dot_num=30
initial_state=kron([1.0 0 0;0 0 0;0 0 0],thermal_state(dim,1))
#########密度矩阵向量化，超算子维数变成n^2*n^2维,态矩阵厄米，只用存一半？意义不大
function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3*dim,:)
end


n=kron(eye(3),a₊(dim)*a₋(dim))
en=kron([0 0 0;0 0 0;0 0 1],eye(dim))
rr=kron([0 0 0;0 1 0;0 0 0],eye(dim))
η1=-0.15
η2=0.15
a=a₊(dim)+a₋(dim)
ag2=kron([0 0 0;0 0 0;1 0 0],exp(im*η1*a))
ag3=kron([0 0 1;0 0 0;0 0 0],exp(-im*η1*a))
ag4=ag2+ag3
ar2=kron([0 0 0;0 0 0;0 1 0],exp(im*η2*a))
ar3=kron([0 0 0;0 0 1;0 0 0],exp(-im*η2*a))
ar4=ar2+ar3

E=eye(3*dim)
γ1=20/3*ν
γ2=40/3*ν

function diss1()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η1*theta*a)
        aup=kron([0 0 0;0 0 0;1 0 0],jump')
        adown=kron([0 0 1;0 0 0;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss2()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η2*theta*a)
        aup=kron([0 0 0;0 0 0;0 1 0],jump')
        adown=kron([0 0 0;0 0 1;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end

dissipation1=γ1*diss1()
dissipation2=γ2*diss2()

# Δg=(Ωr^2+Ωg^2)/ν-ν
# Δr=Δg

function eit_Lindbladian_vec(Ωg,Ωr,Δ)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))+kron(transpose(im*((ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))),E)
    
    return unitary+dissipation1+dissipation2
end

instate=dm2vec(initial_state);


In [5]:
function sweep_detuning_100()
    ttot=100
    initial_x=[5.0 16]
    sweep_num=120
    start=30
    stop=150
    d=[]
    somega_g=[]
    somega_r=[]
    sn=[]
    for i in 1:sweep_num
        println(" ")
        println(i," ")
        m=start+(i-1)*(stop-start)/sweep_num
        function feval(x)
            f=exp(ttot*eit_Lindbladian_vec(x[1],x[2],m))*instate
            nbose=real(tr(n*vec2dm(f)))
            print(" f ",nbose," ")
            return nbose
        end        

        a=optimize(feval,initial_x,LBFGS())
        initial_x=Optim.minimizer(a) 
        append!(somega_g,initial_x[1])  
        append!(somega_r,initial_x[2])
        append!(sn,Optim.minimum(a)[1])
        append!(d,m)
        df1=DataFrame(delta=d,omega_g=somega_g,omega_r=somega_r,n=sn);CSV.write("eit_sweep_detuning_100.csv", df1);
    end
    return somega_g,somega_r,sn,d
end
@time res=sweep_detuning_100()


 
1 
 f 0.6325678416569952  f 0.6325692508908396  f 0.6325788864119263  f 0.6325582061278181  f 0.6325685462583247  f 0.6206393267672379  f 0.6206407567210541  f 0.6206502995157933  f 0.6206297839809335  f 0.620640041728203  f 0.5732828954200766  f 0.5732844139072558  f 0.5732934410262416  f 0.5732738683431216  f 0.573283654645808  f 0.34341420319997756  f 0.3434147577959913  f 0.3434231989996101  f 0.34340576192597805  f 0.3434144804813673  f 0.6818322344114895  f 0.6818485797105769  f 0.6818368309283055  f 0.6818439832852944  f 0.6818404071087989  f 0.11518286010442641  f 0.1151836351216279  f 0.11518305423496418  f 0.11518344105911368  f 0.1151832476013667  f 0.11353124799576102  f 0.1135311090011176  f 0.1135319379399471  f 0.11353041911312961  f 0.11353117848660758  f 0.11303955701500965  f 0.11303970308342412  f 0.11304010983011147  f 0.11303915032714433  f 0.11303963003779775  f 0.10835241874519268  f 0.10835259023976884  f 0.10835307347258576  f 0.10835193554576153  f 0.1083525

 f 0.05904219255217711  f 0.059043122900608085  f 0.05904256003094104  f 0.059042755266928953  f 0.05904265764006339  f 0.058984101034720404  f 0.05898498792173235  f 0.05898445226838289  f 0.05898463653309014  f 0.05898454439195446  f 0.058780639197768984  f 0.05878135306502144  f 0.05878092539543884  f 0.05878106671218916  f 0.058780996045261664  f 0.0584357904508128  f 0.058435648838083804  f 0.058435746433600624  f 0.0584356926949813  f 0.058435719556624285  f 0.05841688459453904  f 0.05841689898528527  f 0.058416901864706154  f 0.05841688155624287  f 0.05841689170272121  f 0.058414740291278  f 0.058414753465911386  f 0.05841475486349687  f 0.05841473873426987  f 0.058414746791090544  f 0.058410739188085745  f 0.058410747160529665  f 0.0584107427140411  f 0.0584107434726449  f 0.05841074308549565  f 0.058508000244230376  f 0.058507973450901835  f 0.05850794188542762  f 0.058508031634435166  f 0.05850798675172989  f 0.05841073347423894  f 0.058410741246455615  f 0.05841073658920448 

 f 0.053658705683676367  f 0.05365966493688972  f 0.053659091163897124  f 0.05365927927088607  f 0.05365918520886655  f 0.053605646076563325  f 0.05360656004342669  f 0.05360601409741311  f 0.05360619183678395  f 0.053606102958657885  f 0.05341989470651912  f 0.053420628274387046  f 0.05342019283192142  f 0.053420329962898354  f 0.05342026138919339  f 0.05310964803712427  f 0.05310948716309271  f 0.05310959138845505  f 0.0531095436195254  f 0.05310956749662819  f 0.05309070330871281  f 0.053090713075835905  f 0.053090715395430255  f 0.053090700798741376  f 0.05309070808958912  f 0.05308967289110678  f 0.053089682714492506  f 0.053089683623617975  f 0.053089671791097685  f 0.053089677699855904  f 0.053087480995869844  f 0.053087490926073996  f 0.053087486220259085  f 0.053087485508842444  f 0.05308748585703365  f 0.05312473109376535  f 0.05312473858153637  f 0.053124706456190586  f 0.05312476301614743  f 0.05312473472841928  f 0.05308741664730418  f 0.053087426576158156  f 0.05308742078

 f 0.05158275224857434  f 0.05158375745098671  f 0.05158316191995794  f 0.05158334755751143  f 0.05158325473032713  f 0.05153183205872865  f 0.05153278830747628  f 0.05153222240484848  f 0.05153239773923787  f 0.05153231006367433  f 0.05135424472447636  f 0.05135500580840126  f 0.05135455768108642  f 0.05135469262924515  f 0.051354625147038306  f 0.05107796028952154  f 0.05107775028321709  f 0.051077879956004435  f 0.05107783038632775  f 0.05107785516391507  f 0.05105255823769681  f 0.05105256459263598  f 0.051052566706522196  f 0.051052555896037966  f 0.05105256129386272  f 0.0510520494981563  f 0.051052056919021774  f 0.051052057533342146  f 0.051052048655706825  f 0.05105205308709907  f 0.051050866891276546  f 0.05105087854855856  f 0.05105087316621971  f 0.05105087204395181  f 0.051050872597635986  f 0.05106589861662814  f 0.051065930779882855  f 0.051065895446441356  f 0.05106593371287754  f 0.051065914572080165  f 0.051050777087563205  f 0.05105079021766364  f 0.05105078273639031

 f 0.05188472096973549  f 0.05188578514274692  f 0.051885159778899875  f 0.05188534606935206  f 0.05188525291567067  f 0.051834145417521606  f 0.051835155395683535  f 0.051834562437733864  f 0.05183473811116252  f 0.05183465026606612  f 0.05165886569467671  f 0.05165965943049675  f 0.051659195417706666  f 0.05165932944246509  f 0.05165926242192851  f 0.0514181797411315  f 0.05141789357917283  f 0.05141806498473656  f 0.05141800806048521  f 0.051418036515296604  f 0.051379969758898435  f 0.05137997382398394  f 0.051379975839387394  f 0.051379967472231336  f 0.05137997164830535  f 0.051379699698297795  f 0.05137970549300408  f 0.05137970592151178  f 0.05137969899824889  f 0.05137970245231901  f 0.0513790402789182  f 0.051379052995256656  f 0.05137904706963968  f 0.0513790459318564  f 0.05137904649316541  f 0.051385985566220864  f 0.05138603296896515  f 0.05138599511190988  f 0.05138602314475048  f 0.05138600912072085  f 0.05137896388139024  f 0.05137897980112408  f 0.05137897093281034  f

...

 f 0.19295834998494055  f 0.192960355574686  f 0.19295922874647  f 0.19295947541210634  f 0.1929593520623181  f 0.19289699426470389  f 0.19289881675119716  f 0.19289779306057905  f 0.19289801655224106  f 0.19289790478950639  f 0.19271069963180917  f 0.19271178848516948  f 0.19271117744689975  f 0.1927113092591114  f 0.19271124333637302  f 0.19319233386367293  f 0.19318972243776023  f 0.19319117856118964  f 0.19319087628945417  f 0.1931910274096346  f 0.19260827001425235  f 0.1926082670444609  f 0.19260826840561102  f 0.1926082672305356  f 0.19260826780174028  f 0.19260826639881914  f 0.19260826700920491  f 0.192608266340672  f 0.1926082656449062  f 0.19260826597646874  f 0.19260827505353645  f 0.19260828998615265  f 0.1926082811979791  f 0.19260828241927766  f 0.19260828179226705  f 0.19260826578326945  f 0.19260826743079876  f 0.19260826617432578  f 0.19260826561724564  f 0.19260826587942576  f 0.19260826508384993  f 0.19260826659869273  f 0.19260826538612602  f 0.1926082648739962  f 

 f 0.20468528234663635  f 0.2046872864286041  f 0.20468615950332  f 0.20468640782019384  f 0.20468628364350977  f 0.20462616650706653  f 0.20462798770146576  f 0.2046269638335208  f 0.2046271889212127  f 0.20462707635913088  f 0.20444663031978222  f 0.20444771868527548  f 0.20444710723421813  f 0.20444724030934086  f 0.20444717375398644  f 0.20491043481623888  f 0.20490782625585677  f 0.2049092823872328  f 0.20490897718294357  f 0.20490912976821773  f 0.20434780970887517  f 0.2043478069988059  f 0.2043478079998664  f 0.2043478072345535  f 0.20434780759969692  f 0.20434780721593013  f 0.20434780754311205  f 0.2043478068275941  f 0.20434780645806572  f 0.2043478066253271  f 0.20434781301739924  f 0.20434782549371955  f 0.20434781791181944  f 0.2043478191258525  f 0.2043478185013732  f 0.2043478070262883  f 0.20434780771095237  f 0.2043478067934555  f 0.20434780647053127  f 0.20434780661441304  f 0.2043478068690901  f 0.20434780750269066  f 0.2043478065985324  f 0.2043478062998948  f 0.20

 f 0.21644100320314064  f 0.21644299999523728  f 0.21644187589754751  f 0.2164421258036282  f 0.21644200083119655  f 0.21638425883338352  f 0.2163860739633989  f 0.21638505230828478  f 0.21638527898947033  f 0.21638516562953125  f 0.216211735456098  f 0.21621282260181585  f 0.21621221099225416  f 0.2162123455584367  f 0.21621227825626715  f 0.21665249232427009  f 0.21664990588556335  f 0.21665135181037146  f 0.21665104485146996  f 0.21665119831261412  f 0.21611610217987012  f 0.2161160999027499  f 0.216116100497705  f 0.2161161000660353  f 0.21611610026281108  f 0.2161161004942516  f 0.21611610060799089  f 0.21611609985590396  f 0.21611609972730617  f 0.21611609977265978  f 0.2161161030935906  f 0.21611611277054074  f 0.2161161066306662  f 0.21611610771462605  f 0.21611610715367807  f 0.21611610046433888  f 0.2161161006373048  f 0.21611609985185523  f 0.21611609973074508  f 0.21611609977237625  f 0.21611610045102306  f 0.21611610061121247  f 0.21611609982763222  f 0.2161160997155852  f

 f 0.22819561087718016  f 0.22819759572733506  f 0.22819647679712285  f 0.22819672826950196  f 0.22819660251244583  f 0.22814129775168046  f 0.22814310296641735  f 0.22814208544270032  f 0.2281423137358564  f 0.22814219956837883  f 0.22797587263143587  f 0.22797695790813266  f 0.22797634637726621  f 0.2279764826147235  f 0.22797641447513828  f 0.22839031885672248  f 0.22838776968222252  f 0.22838919739271585  f 0.22838888955703168  f 0.22838904345496538  f 0.22788308852678096  f 0.22788308686060793  f 0.22788308701099644  f 0.22788308681606445  f 0.2278830868931421  f 0.22788308748274172  f 0.2278830874718715  f 0.2278830866926532  f 0.22788308670180998  f 0.22788308667677595  f 0.2278830876207645  f 0.22788309423116346  f 0.22788308973307164  f 0.22788309055883144  f 0.22788309012537464  f 0.2278830874815241  f 0.22788308747303646  f 0.22788308669251925  f 0.22788308670198731  f 0.22788308667668  f 0.2278830874811748  f 0.22788308747337185  f 0.22788308669273574  f 0.22788308670147286

(Any[10.351145065518676, 10.524138807597753, 10.694576746076972, 10.862535557161, 11.028092220168242, 11.191323187449834, 11.352303754377619, 11.511107890014301, 11.667807888924148, 11.822474283843592  …  22.589288076629877, 22.669282473111792, 22.74896125234803, 22.828329650922264, 22.907392718341168, 22.986155143780316, 23.06462181617023, 23.142797572090945, 23.220686840536445, 23.29829376632204], Any[5.1217575072050385, 5.152038072816614, 5.182995291048952, 5.214589660166535, 5.246777838441348, 5.27951373913029, 5.312749563653989, 5.346436076605457, 5.380523493567517, 5.414961750342487  …  8.229931290218856, 8.257178541844349, 8.284455125770505, 8.311754331993667, 8.3390698243486, 8.366395370772384, 8.393724941615591, 8.421052604993873, 8.44837273255638, 8.475680547146528], Any[0.06617472487623118, 0.0638581518668823, 0.061809676252202156, 0.06000172429479018, 0.05841069469224763, 0.0570162947616585, 0.05580100143498797, 0.054749620943355365, 0.053848927053785224, 0.0530873622231624